In [1]:
import math
import hashlib
import inspect
import json
import pandas as pd
import numpy as np
import re
from hashlib import sha256
from collections import Counter
import string
import os
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import spacy
from spacy.matcher import Matcher
from spacy.lang.en import English
from spacy.matcher import Matcher
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('data/covid19_data.csv')
df_test_pred = pd.read_csv('data/covid19_unlabelled_test.csv')

In [3]:
df_train.shape #(5531, 15)
df_train.head()

,verifiedby,country,class,title,published_date,country1,country2,country3,country4,article_source,ref_source,source_title,content_text,category,lang
0,Delfi Melo Detektorius (Lie Detector),Lithuania,FALSE,Claims that coronavirus is fake and Belarus i...,2020/05/11,Lithuania,NaN,NaN,NaN,https://www.delfi.lt/news/melo-detektorius/mel...,poynter,Baltarusija atšventė nesisaugodama nuo koronav...,Gegužės 9-ąją Baltarusijos sostinėje Minske su...,NaN,lt
1,AfricaCheck,"United States, South Africa",False,Muammar Gaddafi predicted the current coronav...,2020/03/21,United States,South Africa,NaN,NaN,https://africacheck.org/fbcheck/no-gaddafi-did...,poynter,"No, Gaddafi didn’t predict coronavirus, but di...",As the world struggles to bring the new corona...,NaN,en
2,AFP,Hong Kong,False,Video shows quarantined people in a building ...,2020/02/17,Hong Kong,NaN,NaN,NaN,http://u.afp.com/QuarantineChina,poynter,"This video shows tower blocks in Shanghai, not...","It has been viewed more than 200,000 times., T...",NaN,en
3,FactCrescendo,India,False,"A video shows a massive explosion in Wuhan, C...",2020/02/17,India,NaN,NaN,NaN,https://english.factcrescendo.com/2020/02/17/t...,poynter,"This blast took place in Tiajin, China in 2015...","The fact behind every news!, Ever since, the W...",NaN,en
4,Newtral.es,"Spain, Peru, United States, Mexico",False,Chlorine dioxide cures the coronavirus.,2020/03/25,Spain,Peru,United States,Mexico,https://www.newtral.es/bulo-dioxido-de-cloro-c...,poynter,El dióxido de cloro no cura el coronavirus; al...,Circula un video de Youtube titulado: “El Coro...,NaN,es


In [5]:
class_value = df_train['class'].value_counts(dropna=False)

In [6]:
class_value.index
class_value.head()

False           2110
FALSE           2079
Misleading       463
MISLEADING       173
Mostly false     101
Name: class, dtype: int64

In [58]:
#function to classify the label in t(true) or f(false) or r(to revome)
def create_label(doc, list_false, list_true):
    
    if doc in list_false:
        return 'f'
    elif doc in list_true:
        return 't'
    else:
        return 'r'

#to select new data frame with the n_falses quantity of false observations
def create_balance_dataset(df,n_falses,label):
    df = df.copy()
    df = df[['title', label]]
    df_false = df[df[label]=='f'].sample(n=n_falses, random_state=111)
    df_true  = df[df[label]=='t']
    
    return pd.concat([df_false,df_true])       

    

In [60]:
list_false= ['False', 'FALSE']
list_true= ['True','News','Mostly True','MOSTLY TRUE','HALF TRUE','Correct Attribution','Half True','PARTLY TRUE',
            'half true','Correct','Partially true','HALF TRUTH','Partially correct', 'Mostly true','True but','Partly true']

df_train['label'] = df_train['class'].apply(create_label, args=(list_false, list_true))

df_train['label'].value_counts(dropna=False)

f    4189
r    1253
t      89
Name: label, dtype: int64

In [61]:
df_10percTrue = create_balance_dataset(df_train, 801,'label')
print(df_10percTrue.shape)
df_10percTrue['label'].value_counts(dropna=False)

(890, 2)


f    801
t     89
Name: label, dtype: int64

In [62]:
df_10percTrue.head()

,title,label
2871,African Union (AU) has validated COVID-Organi...,f
242,A video of a Muslim fruit-vendor spitting on ...,f
4218,Video shows an alleged hotel owner in New Yor...,f
1602,Post claims U.S. President Donald Trump annou...,f
3711,Images of the medical facility shared with th...,f


In [54]:
df_10percTrue.to_csv('data/df_10percTrue.csv')

In [77]:
#50% true
df_50percTrue = create_balance_dataset(df_train, 89,'label')
print(df_50percTrue.shape)
df_50percTrue['label'].value_counts(dropna=False)
df_50percTrue.to_csv('data/df_50percTrue.csv')

(178, 2)


In [81]:
#removing 'News' as true

list_false= ['False', 'FALSE']
list_true= ['True','Mostly True','MOSTLY TRUE','HALF TRUE','Correct Attribution','Half True','PARTLY TRUE',
            'half true','Correct','Partially true','HALF TRUTH','Partially correct', 'Mostly true','True but','Partly true']

df_train['label_v2'] = df_train['class'].apply(create_label, args=(list_false, list_true))

df_train['label_v2'].value_counts(dropna=False)

df_50percTrue_without_news = create_balance_dataset(df_train, 66,'label_v2')
print(df_50percTrue_without_news.shape)
df_50percTrue_without_news['label_v2'].value_counts(dropna=False)
df_50percTrue_without_news.to_csv('data/df_50percTrue_without_news.csv')

(132, 2)


### second version of true
considering no evidence as true

In [82]:
list_false= ['False', 'FALSE','Misleading','MISLEADING']
list_true = ['True','News','Mostly True','MOSTLY TRUE','HALF TRUE','Correct Attribution','Half True','PARTLY TRUE',
             'half true','Correct','Partially true','HALF TRUTH','Partially correct', 'Mostly true','True but','Partly true',
             'NO EVIDENCE','no evidence','No Evidence','No evidence']

df_train['label_v2'] = df_train['class'].apply(create_label, args=(list_false, list_true))

df_train['label_v2'].value_counts(dropna=False)

f    4825
r     530
t     176
Name: label_v2, dtype: int64

In [85]:
#10% of true observations

df_10percTrue_noevid = create_balance_dataset(df_train, 176*9,'label_v2')
print(df_10percTrue_noevid.shape)
df_10percTrue_noevid['label_v2'].value_counts(dropna=False)

(1760, 2)


f    1584
t     176
Name: label_v2, dtype: int64

In [87]:
df_10percTrue_noevid.to_csv('data/df_10percTrue_with_noevidence.csv')

In [86]:
df_10percTrue_noevid.head()

,title,label_v2
2725,A viral image suggests that cocaine kills cor...,f
1196,Having sex every morning kills coronavirus.,f
3539,Picture shows the Mecca totally empty.,f
3694,Queen Elizabeth is shown in a quote card to b...,f
4519,Official news: in Naples people have used Toc...,f


# 1st submition

In [64]:
sub1 = pd.read_csv('data/submission_1.csv')

In [67]:
sub1.head()

,Unnamed: 0,text,predicted
0,0,American researcher Charles Lieber was arrest...,False
1,1,Did Trump Blame Obama for ‘Bad’ COVID-19 Tests...,False
2,2,Hundreds of sampoerna cigarette factory worke...,False
3,3,Dr. Megha Vyas from Pune died of COVID-19.,False
4,4,Jacob Rothschild owns a patent to coronavirus.,False


In [75]:
sub1['predicted'].replace({False: 'false', True: 'true'})

0       false
1       false
2       false
3       false
4       false
        ...  
2087    false
2088    false
2089    false
2090    false
2091    false
Name: predicted, Length: 2092, dtype: object